In [1]:
notes = [
    ' 2_LINE  5_LINE Uptown trains run express from 3 Av-149 St to E 180 St',
    ' 2_LINE Downtown trains run local from 96 St to Times Sq-42 St',
    ' 2_LINE Flatbush Av-bound trains skip Nereid Av, 233 St, 225 St and 219 St',
    ' 2_LINE trains replace weekend  5_LINE service between Dyre Av and E 180 St',
    ' 4_LINE All trains from Junius St to Sutter Av-Rutland Rd board at the New Lots Av-bound platform',
    ' 4_LINE No trains between Utica Av and New Lots Av',
    ' A_LINE Trains make local stops in both directions at 155 St and 163 St',
    ' D_LINE  N_LINE  R_LINE Prospect Av, 3 Av and 2 Av are closed for renovation',
    ' F_LINE Jamaica-bound trains are rerouted via the  E_LINE after 47-50 Sts to Roosevelt Av',
    ' F_LINE Manhattan-bound platforms at Avenue U, Avenue P, Avenue N, Bay Pkwy  and Avenue I are closed for renovation',
    ' L_LINE Service operates every 12 minutes between Myrtle-Wyckoff Avs and Rockaway Pkwy',
    ' M_LINE Service ends early between 71 Av and Essex St E_LINE  F_LINE  R_LINE trains provide alternate service',
    ' R_LINE Service restored at Bay Ridge Av',
    ' sir_LINE Grant City Station Entrance Closure'
    

]

In [2]:
note_type= [
    'run express',
    'run local',
    'skip',
    'replace',
    'platform change',
    'no train',
    'make stop',
    'close',
    'reroute',
    'platform close',
    'operation frequency',
    'end_early',
    'restore',
    'entrance closure'
    
]

In [3]:
import pandas as pd
cat = pd.DataFrame({'notes': notes, 'type': note_type})
#cat.to_csv('notes&type.csv', index = False)

In [4]:
import re
def parse(a):
    dic = None
    
    ## rerouted
    if 'rerouted' in a:

        a_split = a.split('rerouted')
        first = a_split[0]
        second = a_split[1]

        ##find lines reroute other
        line_a = re.findall('[a-zA-Z0-9]*_LINE', second)

        ##find lines that will be rerouted
        line_b = re.findall('[a-zA-Z0-9]*_LINE', first)

        #direction
        bound = re.findall('_LINE (.*?) trains', first)

        ##between
        stations = re.findall('after (.*?) to (.*)', a)
        start = stations[0][0]
        end = stations[0][1]


        dic = {'type': 'rerout',
               'rerouted by': line_a,
               'rerouting': line_b,
               'from': start,
               'to': end }
    

    elif ('platform' in a and 'closed' in a) or ('platforms' in a and 'closed' in a):

        linebound, rest = re.split('platforms|platform', a)

        #line
        line = re.findall('[a-zA-Z0-9]*_LINE', linebound)

        #direction
        bound = re.findall('[a-zA-Z0-9]*-bound', linebound)

        #reason
        reason = re.findall('for (.*)', rest)

        #stations
        stations_s = re.findall('platform at (.*?) closed|platforms at (.*?) closed ', a)[-1][-1].strip(' are').strip(' is')
        stations = re.split(', | and ',stations_s)

        dic = {'type': 'platform close',
              'direction': bound[0],
              'reason': reason[0],
              'stations':stations,
              'lines': line}  
    
    
    ## close
    elif 'closed' in a:

        ## find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ## stations
        last_station = lines[-1]
        s = last_station + ' (.*?) is|'+ last_station + ' (.*?) are'
        stations_s = re.findall(s, a)[0]
        stations_s = [z for z in stations_s if z != '']
        stations = re.split(', | and ',stations_s[0])


        ## reason 
        reason = re.findall('closed for (.*)', a)[0]

        dic = {'type': 'close',
               'lines': lines,
               'stations': stations,
               'reason': reason}
    
    ## local stops
    elif 'local stops' in a:

        ## find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ## direction
        bound = re.findall('in (.*?) directions', a)[0]

        ## stations
        stations_s = re.findall('local stops (.*?) at (.*)', a)[0][1]
        stations = re.split(', | and ',stations_s)

        dic = {'type': 'local stops',
               'lines': lines,
               'direction': bound ,
               'stations': stations}
    
    
    
    ## platform change
    elif ('platforms' or 'platform' in a) and ('board' in a):

        ##find lines 
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##from to
        stations = re.findall('from (.*?) to (.*) board', a)
        start = stations[0][0]
        end = stations[0][1]

        ##new platform
        platform = re.findall('at the (.*?) platform', a)[0]

        
        dic = {'type': 'change platform',
               'lines': lines,
               'board at': platform,
               'from': start,
               'to': end }
        
        
    ## replace
    elif 'replace' in a:

        a_split = a.split('replace')
        first = a_split[0]
        second = a_split[1]

        ##find lines replacing other
        line_a = re.findall('[a-zA-Z0-9]*_LINE', first)

        ##find lines that will be replaced
        line_b = re.findall('[a-zA-Z0-9]*_LINE', second)

        ##find time 
        time = re.findall('replace (.*?)  [a-zA-Z0-9]*_LINE', a)[-1]

        ##between
        stations = re.findall('between (.*?) and (.*)', a)
        start = stations[0][0]
        end = stations[0][1]


        dic = {'type': 'replace',
               'replaced by': line_a,
               'replacing': line_b,
               'time': time,
               'from': start,
               'to': end }
        
    
    ## skip
    elif 'skip' in a:
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)
        
        ##find bound
        last_line = line[-1]
        text = str(last_line) + r' (.*?) trains'
        bounds = re.findall(text, a)
        if bounds == []:
            bound = []
        else:
            bound = bounds[0]
        
        ##Stations
        stations_s = re.findall('skip (.*)', a)
        stations = re.split(', | and ',stations_s[0])
        
        dic = {'type': 'skip',
               'lines': line,
               'direction': bound,
               'stations': stations}
     

        
    ## run express
    elif 'run express' in a:
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##find bound
        last_line = line[-1]
        text = str(last_line) + r' (.*?) trains'
        bound = re.findall(text, a)[0]

        ##station from 
        start = re.findall(r'from (.*?) to', a)

        ##station to
        end = re.findall('to (.*)', a)
        

        dic = {'type': 'run express',
               'lines': line,
               'direction': bound,
               'from': start,
               'to': end}
    
    ## run local
    elif 'run local' in a:
        
        if 'via' in a:
            
            ##split the sentence 
            first, second = a.split('via')
            
            ##find lines 
            line = re.findall('[a-zA-Z0-9]*_LINE', first)
            
            ##via
            via = re.findall('[a-zA-Z0-9]*_LINE', first)
            
            ##find bound
            last_line = line[-1]
            text = str(last_line) + r' (.*?) trains'
            bound = re.findall(text, first)[0]

            ##station from 
            start = re.findall(r'from (.*?) to', second)

            ##station to
            end = re.findall('to (.*)', second)
            
            dic = {'type': 'run local',
                   'lines': line,
                   'via' : via,
                   'direction': bound,
                   'from': start,
                   'to': end}
                      
        else:
            ##find lines
            line = re.findall('[a-zA-Z0-9]*_LINE', a)

            ##find bound
            last_line = line[-1]
            text = str(last_line) + r' (.*?) trains'
            bound = re.findall(text, a)[0]

            ##station from 
            start = re.findall(r'from (.*?) to', a)

            ##station to
            end = re.findall('to (.*)', a)
            
            ##via?
            via = 'itself'
            
            dic = {'type': 'run local',
                   'lines': line,
                   'via' : via,
                   'direction': bound,
                   'from': start,
                   'to': end}

        
    ## operation frequency    
    elif 'operates every' in a:
        
        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE Service', a)
        line = [x.strip(' Service') for x in line]

        ##frequency
        freq = re.findall('every (.*) between',a)

        ##start from
        start = re.findall(r'between (.*?) and', a)

        ##end to
        end = re.findall('%s and (.*)'%start[0], a)

        dic = {'type' : 'operate frequency',
              'lines' : line,
              'frequency' : freq[0],
              'from' : start[0],
              'to' : end[0]}
        
        
    ## Service end early
    elif 'ends early' in a or 'end early' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE Service', a)
        line = [x.strip(' Service') for x in line]

        ##No direction

        ##start from
        start = re.findall(r'between (.*?) and', a)

        ##alternative lines
        s_2 = a.replace(line[0],'')
        alt_lines = re.findall('[a-zA-Z0-9]*_LINE', s_2)
        
        ##end to
        end = re.findall('and (.*) %s'%alt_lines[0], a)

        dic = {'lines': line,
               'from': start[0],
               'to' : end,
               'alt lines' : alt_lines,
               'type' : 'end early'}    
    
    
    ## NO TRAIN
    elif 'No trains' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        if 'running' in a:
            start = 'all stations'
            end = 'all stations'
            
        else:
            ##station from
            start = re.findall(r'between (.*?) and', a)[0]

            ##station to
            end = re.findall('and (.*)', a)[0]

        dic = {'lines' : line,
               'from': start,
               'to': end,
              'type': 'No train'}
     
    ## ENTRANCE CLOSURE
    elif 'Entrance Closure' in a:

        ##find lines
        line = re.findall('[a-zA-Z0-9]*_LINE', a)

        ##stop
        stop = re.findall('_LINE (.*?) Station', a)

        dic = {'lines' : line,
                'station' : stop,
                'type' : 'Entrance Closure'}
 
   
    ## restore
    elif 'restored' in a:
        
        ##find lines
        lines = re.findall('[a-zA-Z0-9]*_LINE', a)
        
        ##stations
        stations_s = re.findall('at (.*)', a)[0]
        stations = re.split(', | and ',stations_s)
        
        dic = {'lines' : lines,
               'station' : stations,
               'type' : 'restore'}

    if dic == None:
        print("no category found")
        return None
    else:
        return dic

In [5]:
ls = [' 1_LINE Van Cortlandt Park-bound trains skip 50 St, 59 St and 66 St',
 ' 1_LINE South Ferry-bound trains skip 28, 23, 18, Christopher, Houston, Canal and Franklin Sts',
 ' 3_LINE No trains running',
 'No elevator service to/from the DOWNTOWN platform at Canal St  6_LINE Station',
 ' 6_LINE Downtown trains run express from 125 St to Brooklyn Bridge',
 ' 6_LINE Service operates to/from Bowling Green',
 'No elevator service to/from the Flushing-bound platform at 61 St-Woodside  7_LINE Station',
 ' A_LINE Ozone Park/Far Rockaway-bound trains skip 163 St, 155 St and 135 St',
 ' A_LINE Ozone Park/Far Rockaway-bound trains run express from Hoyt Schermerhorn to Euclid Av',
 ' A_LINE Ozone Park/Far Rockaway-bound trains skip 50 St, 23 St and Spring St',
 ' C_LINE Euclid Av-bound trains run express from Hoyt Schermerhorn to Euclid Av',
 ' D_LINE  N_LINE  R_LINE Service restored at Prospect Av',
 ' D_LINE Coney Island-bound trains skip 182-183 Sts',
 ' D_LINE Norwood-bound trains run local via the  A_LINE from W 4 St to 59 St-Columbus Circle',
 ' E_LINE Jamaica Center-bound trains skip 75 Av and Briarwood',
 ' E_LINE World Trade Center-bound trains skip 23 St and Spring St',
 ' F_LINE Manhattan-bound platforms at Avenue U, Avenue P, Avenue N, Bay Pkwy  and Avenue I are closed for renovation',
 ' F_LINE Jamaica-bound trains skip 75 Av, Briarwood and Sutphin Blvd',
 ' J_LINE  Z_LINE Manhattan-bound platform at 104 St is closed for renovation',
 ' J_LINE  Z_LINE Manhattan-bound platform at 121 St is closed for renovation',
 'VIADUCT RECONSTRUCTION M_LINE No trains between Myrtle-Wyckoff Av and Myrtle Av (Broadway)',
 ' M_LINE Service ends early between 71 Av and Essex St E_LINE  F_LINE  R_LINE trains provide alternate service',
 ' N_LINE Coney Island-bound platforms at Fort Hamilton Pkwy, New Utrecht Av, 18 Av, 20 Av, Kings Hwy, Avenue U and 86 St are closed for renovation',
 ' N_LINE  W_LINE 30 Av and 36 Av are closed for renovation',
 ' R_LINE Forest Hills-bound trains skip Elmhurst, Grand Avs, Woodhaven Blvd, 63 Dr and 67 Av',
 ' sir_LINE Fallen leaves may slow service']

In [6]:
ind = 0
n = 0
for i in ls:
    print(ind)
    ind += 1
    print(i)
    if parse(i) == None:
        n +=1
    print(parse(i))
n

0
 1_LINE Van Cortlandt Park-bound trains skip 50 St, 59 St and 66 St
{'type': 'skip', 'lines': ['1_LINE'], 'direction': 'Van Cortlandt Park-bound', 'stations': ['50 St', '59 St', '66 St']}
1
 1_LINE South Ferry-bound trains skip 28, 23, 18, Christopher, Houston, Canal and Franklin Sts
{'type': 'skip', 'lines': ['1_LINE'], 'direction': 'South Ferry-bound', 'stations': ['28', '23', '18', 'Christopher', 'Houston', 'Canal', 'Franklin Sts']}
2
 3_LINE No trains running
{'lines': ['3_LINE'], 'from': 'all stations', 'to': 'all stations', 'type': 'No train'}
3
No elevator service to/from the DOWNTOWN platform at Canal St  6_LINE Station
no category found
no category found
None
4
 6_LINE Downtown trains run express from 125 St to Brooklyn Bridge
{'type': 'run express', 'lines': ['6_LINE'], 'direction': 'Downtown', 'from': ['125 St'], 'to': ['Brooklyn Bridge']}
5
 6_LINE Service operates to/from Bowling Green
no category found
no category found
None
6
No elevator service to/from the Flushing-bo

4

In [7]:
notes[7]

' D_LINE  N_LINE  R_LINE Prospect Av, 3 Av and 2 Av are closed for renovation'

In [8]:
parse(notes[7])

{'lines': ['D_LINE', 'N_LINE', 'R_LINE'],
 'reason': 'renovation',
 'stations': ['Prospect Av', '3 Av', '2 Av'],
 'type': 'close'}